test cases:

1. get the expected output
2. can throw out errors (unfitted, format error, etc.)
3. produce correct (or nearly correct) amount of NAs

In [1]:
import pandas as pd
import numpy as np

In [2]:
%cd /Users/alex/PETsARD

/Users/alex/PETsARD


In [3]:
import PETsARD

In [4]:
loader = PETsARD.Loader(filepath = '[NHANES] B.csv',
                                header_exist = False,
                                header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [5]:
df = loader.data

In [6]:
m = PETsARD.Metadata()
m.build_metadata(df)

In [7]:
metadata = {'col': {
                'gen': {'type': 'categorical', 'na_percentage': 0.0},
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'edu': {'type': 'categorical', 'na_percentage': 0.0},
                'mar': {'type': 'categorical', 'na_percentage': 0.0},
                'bmi': {'type': 'numerical', 'na_percentage': 0.0},
                'dep': {'type': 'numerical', 'na_percentage': 0.0},
                'pir': {'type': 'numerical', 'na_percentage': 0.0},
                'gh': {'type': 'numerical', 'na_percentage': 0.0},
                'mets': {'type': 'numerical', 'na_percentage': 0.0},
                'qm': {'type': 'categorical', 'na_percentage': 0.0},
                'dia': {'type': 'numerical', 'na_percentage': 0.0}
                },
             'global':{
                 'row_num': 4190,
                 'col_num': 12,
                 'na_percentage': 0.0
                }
            }

In [8]:
pm = PETsARD.Processor.Base.Processor(m)

In [9]:
from PETsARD.Processor.Missingist import *
from PETsARD.Processor.Outlierist import *
from PETsARD.Processor.Encoder import *
from PETsARD.Processor.Scaler import *

In [10]:
# df.loc[[1,2,3,4,5], 'gen'] = np.nan
# df.loc[[10,20,30,40,5], 'age'] = np.nan

In [11]:
# test global outlierist transformation

config_1 = {'missingist': {'gen': Missingist_Drop(), 'age': Missingist_Simple(5)},
            'outlierist': {'gen': None, 'age': Outlierist_ZScore()},
            'encoder': {'gen': Encoder_Label()},
            'scaler': {'age': Scaler_Log()}}

In [12]:
config_2 = {'missingist': {'gen': Missingist_Drop(), 'age': 'missingist_simple'},
            'outlierist': {'gen': None, 'age': 'outlierist_lof'},
            'encoder': {'gen': 'encoder_uniform'},
            'scaler': {'age': 'scaler_minmax'}}

In [13]:
pm.update_config(config_2)

In [14]:
pm.get_config()

{'missingist': {'gen': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284a8fd0>,
  'age': <PETsARD.Processor.Missingist.Missingist_Simple at 0x1284a8640>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284aafe0>,
  'edu': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284aaf20>,
  'mar': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284aae00>,
  'bmi': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1040ffe20>,
  'dep': <PETsARD.Processor.Missingist.Missingist_Mean at 0x103eff6a0>,
  'pir': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284aada0>,
  'gh': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284aab60>,
  'mets': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284aa470>,
  'qm': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284ab5b0>,
  'dia': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284aab00>},
 'outlierist': {'gen': None,
  'age': <PETsARD.Processor.Outlierist.Outlierist_LOF at 0x1284ab1f0>,
  'race': None,
  '

In [15]:
pm.fit(df)

In [16]:
df_transformed = pm.transform(df)

In [17]:
df_transformed

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,0.983984,0.700000,0.188138,0.355015,0.381214,-0.202041,-0.514504,-0.517100,0.0,0.0,0.336251,1.949241
1,0.673244,0.550000,0.321401,0.536103,0.748729,0.235906,-0.514504,1.933861,0.0,0.0,0.004694,-0.513020
2,0.829190,0.966667,0.209097,0.744981,0.389733,-0.056059,-0.514504,-0.517100,0.0,0.0,0.957835,1.949241
3,0.214191,0.600000,0.145821,0.414572,0.832810,1.929305,1.943618,-0.517100,0.0,0.0,0.890496,-0.513020
4,0.268213,0.366667,0.347538,0.121564,0.753166,-1.296911,1.943618,-0.517100,0.0,0.0,0.592974,-0.513020
...,...,...,...,...,...,...,...,...,...,...,...,...
4162,0.899865,0.083333,0.735313,0.455991,0.576789,-1.194723,-0.514504,-0.517100,0.0,0.0,0.126429,-0.513020
4163,0.263538,0.933333,0.235960,0.630655,0.908662,-1.121732,1.943618,-0.517100,0.0,0.0,0.180097,-0.513020
4164,0.374200,1.000000,0.181959,0.186634,0.927066,0.265103,-0.514504,-0.517100,0.0,0.0,0.115808,-0.513020
4165,0.635423,0.250000,0.090404,0.880763,0.303570,-0.464810,-0.514504,1.933861,0.0,0.0,0.255565,-0.513020


In [18]:
df_inverse = pm.inverse_transform(df_transformed)

In [19]:
df_inverse

,gen,age,race,edu,mar,bmi,dep,pir,gh,mets,qm,dia
0,Male,62.000004,White,Graduate,Married,27.799999,0.0,2.775558e-17,0.0,0.0,Q2,1.000000e+00
1,Male,53.000004,White,HighSchool,Divorced,30.799999,0.0,1.000000e+00,0.0,0.0,Q1,2.775558e-17
2,Male,78.000000,White,HighSchool,Married,28.799999,0.0,2.775558e-17,0.0,0.0,Q3,1.000000e+00
3,Female,56.000000,White,Graduate,Parther,42.400002,1.0,2.775558e-17,0.0,0.0,Q3,2.775558e-17
4,Female,42.000000,Black,College,Divorced,20.299999,1.0,2.775558e-17,0.0,0.0,Q4,2.775558e-17
...,...,...,...,...,...,...,...,...,...,...,...,...
4162,Male,25.000000,Other,Graduate,Never,21.000000,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
4163,Female,76.000000,White,HighSchool,Widowed,21.500000,1.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
4164,Female,80.000000,White,College,Widowed,31.000000,0.0,2.775558e-17,0.0,0.0,Q1,2.775558e-17
4165,Male,35.000000,White,9th,Married,26.000000,0.0,1.000000e+00,0.0,0.0,Q1,2.775558e-17


In [20]:
try:
    print(f'Inverse successful:\n{(df_inverse == df).all()}')
    print(f'Numeric close (age): {np.isclose(df_inverse.age, df.age).all()}')
    print(f'Numeric close (bmi): {np.isclose(df_inverse.bmi, df.bmi).all()}')
except:
    print('Not applicable.')

Not applicable.


In [21]:
loader2 = PETsARD.Loader(filepath = '[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader2.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [22]:
df2 = loader2.data

In [23]:
metadata2 = {'metadata_col': {
                'age': {'type': 'numerical', 'na_percentage': 0.0},
                'workclass': {'type': 'categorical', 'na_percentage': 0.057307},
                'fnlwgt': {'type': 'numerical', 'na_percentage': 0.0},
                'education': {'type': 'categorical', 'na_percentage': 0.0},
                'educational-num': {'type': 'numerical', 'na_percentage': 0.0},
                'marital-status': {'type': 'categorical', 'na_percentage': 0.0},
                'occupation': {'type': 'categorical', 'na_percentage': 0.057512},
                'relationship': {'type': 'categorical', 'na_percentage': 0.0},
                'race': {'type': 'categorical', 'na_percentage': 0.0},
                'gender': {'type': 'categorical', 'na_percentage': 0.0},
                'capital-gain': {'type': 'numerical', 'na_percentage': 0.0},
                'capital-loss': {'type': 'numerical', 'na_percentage': 0.0},
                'hours-per-week': {'type': 'numerical', 'na_percentage': 0.0},
                'native-country': {'type': 'categorical', 'na_percentage': 0.017546},
                'income': {'type': 'categorical', 'na_percentage': 0.0}
                },
             'metadata_global':{
                 'row_num': 48842,
                 'col_num': 15,
                 'na_percentage': 0.07411653904426518
                }
            }

In [24]:
m2 = PETsARD.Metadata()
m2.build_metadata(df2)

In [25]:
pm2 = PETsARD.Processor.Base.Processor(m2)

In [26]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854b9d0>,
  'workclass': <PETsARD.Processor.Missingist.Missingist_Drop at 0x12854b970>,
  'fnlwgt': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854be20>,
  'education': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549780>,
  'educational-num': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284ab460>,
  'marital-status': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549000>,
  'occupation': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284a94e0>,
  'relationship': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549ba0>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1285495a0>,
  'gender': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128548a30>,
  'capital-gain': <PETsARD.Processor.Missingist.Missingist_Mean at 0x128549a20>,
  'capital-loss': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854b4f0>,
  'hours-per-week': <PETsARD.Processor.Missing

In [27]:
pm2.fit(df2)

In [28]:
pm2.get_config()

{'missingist': {'age': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854b9d0>,
  'workclass': <PETsARD.Processor.Missingist.Missingist_Drop at 0x12854b970>,
  'fnlwgt': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854be20>,
  'education': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549780>,
  'educational-num': <PETsARD.Processor.Missingist.Missingist_Mean at 0x1284ab460>,
  'marital-status': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549000>,
  'occupation': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1284a94e0>,
  'relationship': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128549ba0>,
  'race': <PETsARD.Processor.Missingist.Missingist_Drop at 0x1285495a0>,
  'gender': <PETsARD.Processor.Missingist.Missingist_Drop at 0x128548a30>,
  'capital-gain': <PETsARD.Processor.Missingist.Missingist_Mean at 0x128549a20>,
  'capital-loss': <PETsARD.Processor.Missingist.Missingist_Mean at 0x12854b4f0>,
  'hours-per-week': <PETsARD.Processor.Missing

In [29]:
df_transformed2 = pm2.transform(df2)

In [30]:
df_transformed2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,-0.995129,0.430167,0.351675,0.843465,-1.197259,0.681103,0.750644,0.732666,0.904105,0.548921,-0.144804,-0.217127,-0.034087,0.719135,0.059984
1,-0.046942,0.536805,-0.945524,0.085899,-0.419335,0.325506,0.915161,0.022099,0.230007,0.476964,-0.144804,-0.217127,0.772930,0.129746,0.132938
2,-0.776316,0.821474,1.394723,0.852132,0.747550,0.039733,0.982472,0.293247,0.018831,0.462414,-0.144804,-0.217127,-0.034087,0.156960,0.878963
3,-1.068066,0.000359,1.704525,0.381573,-0.030373,0.664701,0.676733,0.895639,0.116907,0.719370,-0.144804,-0.217127,-0.034087,0.804822,0.677721
4,-0.192816,0.996292,0.215911,0.673704,1.136512,0.308334,0.477544,0.182961,0.109659,0.556751,-0.144804,-0.217127,-0.034087,0.650554,0.044706
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,1.047121,0.037359,1.251867,0.734740,1.525474,0.269778,0.305716,0.167850,0.265118,0.274917,-0.144804,-0.217127,-0.034087,0.869390,0.841794
26929,-1.213941,0.530113,1.140952,0.355992,-0.030373,0.591518,0.992250,0.577286,0.005160,0.414875,-0.144804,-0.217127,-0.034087,0.906551,0.236538
26930,-0.849254,0.363047,0.640492,0.868716,0.747550,0.103290,0.950934,0.967775,0.658251,0.795935,-0.144804,-0.217127,-0.195490,0.439498,0.132694
26931,0.098933,0.144073,-0.334178,0.267697,-0.419335,0.356887,0.805329,0.072539,0.737837,0.048032,-0.144804,-0.217127,-0.034087,0.654963,0.920859


In [31]:
df_inverse2 = pm2.inverse_transform(df_transformed2)

In [32]:
df_inverse2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,24.0,Private,369667.0,Some-college,10.0,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
4,36.0,NaN,212465.0,Bachelors,13.0,Married-civ-spouse,NaN,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26928,53.0,Private,321865.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
26929,22.0,Private,310152.0,Some-college,10.0,Never-married,Protective-serv,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
26930,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
26931,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,NaN,Husband,White,Male,0.0,0.0,40.0,NaN,>50K


In [33]:
df2

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [34]:
df_inverse2.isna().mean(axis=0)

age                0.000000
workclass          0.057290
fnlwgt             0.000000
education          0.000000
educational-num    0.000000
marital-status     0.000000
occupation         0.057476
relationship       0.000000
race               0.000000
gender             0.000000
capital-gain       0.000000
capital-loss       0.000000
hours-per-week     0.000000
native-country     0.017525
income             0.000000
dtype: float64

In [35]:
df_inverse2.isna().any(axis=1).mean()

0.07136226933501652

In [36]:
pm.get_changes()

,processor,col,current,default
0,missingist,age,Missingist_Simple,Missingist_Mean
1,outlierist,gen,Outlierist_LOF,NoneType
2,outlierist,age,Outlierist_LOF,Outlierist_IQR
3,outlierist,race,Outlierist_LOF,NoneType
4,outlierist,edu,Outlierist_LOF,NoneType
5,outlierist,mar,Outlierist_LOF,NoneType
6,outlierist,bmi,Outlierist_LOF,Outlierist_IQR
7,outlierist,dep,Outlierist_LOF,Outlierist_IQR
8,outlierist,pir,Outlierist_LOF,Outlierist_IQR
9,outlierist,gh,Outlierist_LOF,Outlierist_IQR


In [37]:
pm._default_processor['outlierist']['categorical']

<function PETsARD.Processor.Base.Processor.__init__.<locals>.<lambda>()>

In [38]:
pm._default_processor['outlierist']['numerical']

PETsARD.Processor.Outlierist.Outlierist_IQR

In [39]:
from sdv.metadata import SingleTableMetadata

In [40]:
sdvmd = SingleTableMetadata()

In [41]:
sdvmd.detect_from_dataframe(df)

In [42]:
t = sdvmd.to_dict()

In [43]:
sdvmd2 = SingleTableMetadata()

In [44]:
mm = m.to_sdv()